# Recomendador KNN

## Inicializacion

In [1]:
from sklearn.metrics import accuracy_score
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from random import shuffle
import numpy as np
import pandas as pd 
import matplotlib.pylab as plt
import os
 
# from google.colab import drive

In C:\Users\manoc\Anaconda3\envs\L_Datos\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\manoc\Anaconda3\envs\L_Datos\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\manoc\Anaconda3\envs\L_Datos\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\manoc\Anaconda3\envs\L_Datos\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\manoc\Anaconda3\envs\L_Datos

In [3]:
# drive.mount('/content/drive')

Mounted at /content/drive


## Carga de datos

In [2]:
# tp_dir = ''
# os.chdir(tp_dir)
os.listdir()

['anime.csv',
 'anime_filtrado.csv',
 'Anime_Synopsis2Genere.ipynb',
 'Anime_Topic_Analisis.ipynb',
 'anime_with_synopsis.csv',
 'dataframe_anime_1.csv',
 'Dataframe_anime_1_10000_V1.csv',
 'Dataframe_anime_1_5000_V1.csv',
 'manipular_dataset.ipynb',
 'Recomendador_KNN.ipynb']

In [4]:
# df = pd.read_csv('dataframe_anime_1.csv')
# df = pd.read_csv('Dataframe_anime_1_10000_V1.csv')
df = pd.read_csv('Dataframe_anime_1_5000_V1.csv')
df.reset_index(inplace=True)
df.head()

KeyboardInterrupt: 

In [7]:
df_general = pd.read_csv('anime.csv')[['MAL_ID',    'Name', 'English name', 'Genders', 'Score']]
df_general.head()

,MAL_ID,Name,English name,Genders,Score
0,1,Cowboy Bebop,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",8.78
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop:The Movie,"Action, Drama, Mystery, Sci-Fi, Space",8.39
2,6,Trigun,Trigun,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",8.24
3,7,Witch Hunter Robin,Witch Hunter Robin,"Action, Mystery, Police, Supernatural, Drama, ...",7.27
4,8,Bouken Ou Beet,Beet the Vandel Buster,"Adventure, Fantasy, Shounen, Supernatural",6.98


In [8]:
# X = df.drop(columns=df.columns[0]).values
X = df.values
# df[df.columns[0]]
y = X[:,0]

Separamos los datos en grupo de entrenamiento y grupo de testeo

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(5000, 17563)
(3000, 17563)
(2000, 17563)


In [ ]:
# df.set_index('Unnamed: 0', inplace=True)

In [11]:
# X_test[:,0].shape#.reshape(-1,)
df_train = df.iloc[X_train[:,0].reshape(-1,)]
 
df_test = df.iloc[X_test[:,0].reshape(-1,)]

## Recomendador KNN
Definimos la funcion del modelo que efectivamente se encargara de realizar las recomendaciones 

In [12]:
viewed = []
n = 1
user_total = X_test[n,1:]
 
pd.set_option("display.max_rows", None, "display.max_columns", None)
 
def KNN_recomender(user_total, X_train, MAL_IDs,k_neighbors = 10, print_my_likes=False, max_print=20,IDs2Title=df_general, Title_key='Name'):
    # X_train has liked (1) and disliked (-1) and not watched (0), lo mismo para user_total
    X_v = X_train[:,0].reshape(-1,1)
    user_v = np.array([user_total[0]])
    viewed = []
    viewed_int = []
    for i in range(0, len(X_train[0,:])):
        if user_total[i]!=0:
            viewed.append(MAL_IDs[i])
            viewed_int.append(int(MAL_IDs[i]))
            user_v = np.append(user_v, user_total[i])
            X_v = np.concatenate((X_v, X_train[:,i].reshape(-1,1)), axis=1)
    user_v = user_v[1:] 
    X_v = X_v[:,1:]
    
    model_knn = NearestNeighbors(
              # metric='cosine',
              n_neighbors=k_neighbors)
    model_knn.fit(X_v)
 
    # clf = KNeighborsClassifier(n_neighbors=k_neighbors) # clasificador
    # clf.fit(X_v, X_v[:,0])
 
    if (print_my_likes==True):
        df_viewed = pd.merge(df_general, pd.DataFrame({'MAL_ID': viewed_int, 'liked?':user_v}), how='inner', on='MAL_ID')
        print(df_viewed.head(max_print))
        print(df_viewed.shape)
 
 
    dist, knn_index = model_knn.kneighbors(user_v.reshape(1, -1) , return_distance=True) # esto me devuelve los indices de los K samples mas cercanos
    knn_index = knn_index[0]
    # knn_index
    recomendations = []
    recom_value = []
    count = []
    for kn in knn_index:
        for i in range(1, len(X_train[0,:])):
            if X_train[kn,i]!=0 and (MAL_IDs[i] not in viewed):
                if str(i) not in recomendations:
                    recomendations.append(MAL_IDs[i])
                    recom_value.append(X_train[kn,i])#/k_neighbors)
                    count.append(1)
                else:
                  # if i in recomendations:
                    recom_value[np.where(np.array(recomendations)==str(i))[0][0]] += X_train[kn,i]#/k_neighbors
                    count[np.where(np.array(recomendations)==str(i))[0][0]] += 1
    for i in range(len(recomendations)):
        recomendations[i] = int(recomendations[i])
        recom_value[i] = recom_value[i]/count[i]
        count[i] = count[i]/k_neighbors
    return recomendations, recom_value, count
            
 
recomendations, recom_value, rel_count = KNN_recomender(user_total, X_train[:,1:], df.columns[1:], print_my_likes=True)
 
# print(recomendations)
# print(recom_value)
 
df_recomendations = pd.merge(df_general, pd.DataFrame({'MAL_ID': recomendations, 'recom_value': recom_value, 'rel_count': rel_count,
                                                       'recom_value_abs': np.array(recom_value)*np.array(rel_count)}),
                                                        how='inner', on='MAL_ID')
df_recomendations.sort_values(by=[
                                  # 'recom_value', 
                                  'recom_value_abs',
                                  'rel_count'
                                  ], ascending=False, inplace=True)
 
df_recomendations.head(10)

    MAL_ID                                             Name  \
0       20                                           Naruto   
1      121                              Fullmetal Alchemist   
2      136                                  Hunter x Hunter   
3      137        Hunter x Hunter: Original Video Animation   
4      138                    Hunter x Hunter: Greed Island   
5      149                                         Loveless   
6      154                                      Shaman King   
7      164                                    Mononoke Hime   
8      199                    Sen to Chihiro no Kamikakushi   
9      228                                    Jigoku Shoujo   
10     235                                  Detective Conan   
11     249                                         InuYasha   
12     270                                         Hellsing   
13     391                 Tales of Phantasia The Animation   
14     405                          Blood: The Last Vam

,MAL_ID,Name,English name,Genders,Score,recom_value,rel_count,recom_value_abs
999,4970,Afro Samurai: Resurrection,Afro Samurai:Resurrection,"Action, Adventure, Samurai",7.36,1.0,0.8,0.8
2530,14199,Oniichan dakedo Ai sae Areba Kankeinai yo ne!,OniAi,"Comedy, Ecchi, Harem, Romance, School",6.53,1.0,0.8,0.8
126,245,Great Teacher Onizuka,Great Teacher Onizuka,"Slice of Life, Comedy, Drama, School, Shounen",8.7,1.0,0.7,0.7
1417,7375,Shakugan no Shana-tan S,Unknown,"Comedy, Parody",6.9,1.0,0.7,0.7
1863,10156,Sacred Seven,Sacred Seven,"Action, Sci-Fi, Super Power, School",6.63,1.0,0.7,0.7
2612,15039,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada...,anohana:The Flower We Saw That Day The Movie,"Slice of Life, Supernatural, Drama",7.95,1.0,0.7,0.7
783,3577,Kyouran Kazoku Nikki,Unknown,"Comedy, Sci-Fi",7.28,1.0,0.6,0.6
1292,6594,Katanagatari,Katanagatari,"Action, Adventure, Historical, Martial Arts, R...",8.36,1.0,0.6,0.6
1330,6773,Shakugan no Shana III (Final),Shakugan no Shana:Season III,"Action, Supernatural, Drama, Romance, Fantasy",7.56,1.0,0.6,0.6
2138,11339,A-Channel+smile,Unknown,"Comedy, School, Seinen, Slice of Life",7.12,1.0,0.6,0.6


In [13]:
 
 
df_recomendations[df_recomendations['rel_count']>1].head(10)

,MAL_ID,Name,English name,Genders,Score,recom_value,rel_count,recom_value_abs


Armo un buscador para definir que animes me gustan ycuales no

In [30]:
name = 'Your Name'
df_general[np.logical_or(df_general['Name'].str.contains(name),
                         df_general['English name'].str.contains(name))].head(20) #Better way of doing the same thing

,MAL_ID,Name,English name,Genders,Score
11308,32281,Kimi no Na wa.,Your Name.,"Romance, Supernatural, School, Drama",8.96


In [29]:
df2 = df.append(df.iloc[0], ignore_index=True)
for col in df.columns: # Vacio la nueva fila
    df2[col][1000]=0
 
# Agrego un caso de ejemplo 

df2['45'][1000]=1           # Rurouni Kenshin
df2['16498'][1000]=1        # Attack on Titan
df2['813'][1000]=1          # Dragon Ball Z
df2['501'][1000]=-1         # Doraemon
df2['200'][1000]=-1         # Cheeky Angel
df2['32281'][1000]=1        # Your Name.
df2['199'][1000]=1          # Spirited Away
df2['164'][1000]=1          # Princess Mononoke
 
# print(df2.head())
# df2.reset_index(inplace=True)
# print(df2.head())
X_user = df2.values[1000,1:]
X_train[:,1:].shape
 
# df_general[df_general['Name'].str.contains('Rurouni')].head(100) #Better way of doing the same thing
 
# df_general[df_general['English name'].str.contains('Rurouni')].head(100) #Better way of doing the same thing

(3000, 17562)

In [25]:
# Aora voy a aplicar el recomendador a mis datos
recomendations, recom_value, rel_count = KNN_recomender(X_user, X_train[:,1:], df.columns[1:],k_neighbors=10, print_my_likes=True)
 
df_recomendations = pd.merge(df_general, pd.DataFrame({'MAL_ID': recomendations, 'recom_value': recom_value, 'rel_count': rel_count,
                                                       'recom_value_abs': np.array(recom_value)*np.array(rel_count)}),
                                                        how='inner', on='MAL_ID')
df_recomendations.sort_values(by=[
                                  # 'recom_value', 
                                  'recom_value_abs',
                                  'rel_count'
                                  ],
                                  ascending=False,
                                  inplace=True)
 
df_recomendations.head(20)

    MAL_ID                                     Name  \
0       20                                   Naruto   
1       22                      Tennis no Ouji-sama   
2       45  Rurouni Kenshin: Meiji Kenkaku Romantan   
3      199            Sen to Chihiro no Kamikakushi   
4      200                      Tenshi na Konamaiki   
5      501                                 Doraemon   
6      813                            Dragon Ball Z   
7     1535                               Death Note   
8     1735                       Naruto: Shippuuden   
9    16498                       Shingeki no Kyojin   
10   16870               The Last: Naruto the Movie   
11   32281                           Kimi no Na wa.   
12   34566          Boruto: Naruto Next Generations   

                      English name  \
0                           Naruto   
1             The Prince of Tennis   
2                  Rurouni Kenshin   
3                    Spirited Away   
4                     Cheeky Angel   
5

,MAL_ID,Name,English name,Genders,Score,recom_value,rel_count,recom_value_abs
1560,15199,Code Geass: Boukoku no Akito 3 - Kagayaku Mono...,Code Geass:Akito the Exiled - The Brightness F...,"Action, Mecha, Military",7.33,0.750000,0.8,0.6
84,142,Kamikaze Kaitou Jeanne,Unknown,"Adventure, Comedy, Demons, Drama, Fantasy, Mag...",7.45,1.000000,0.6,0.6
319,770,Pale Cocoon,Unknown,"Drama, Sci-Fi",7.37,0.714286,0.7,0.5
141,239,Gankutsuou,Gankutsuou:The Count of Monte Cristo,"Drama, Mystery, Sci-Fi, Supernatural, Thriller",8.17,1.000000,0.5,0.5
370,900,Dragon Ball Z Movie 07: Kyokugen Battle!! Sand...,Dragon Ball Z:Super Android 13!,"Action, Adventure, Comedy, Fantasy, Sci-Fi, Sh...",6.89,1.000000,0.5,0.5
1117,9756,Mahou Shoujo Madoka★Magica,Puella Magi Madoka Magica,"Psychological, Drama, Magic, Thriller",8.38,1.000000,0.5,0.5
144,245,Great Teacher Onizuka,Great Teacher Onizuka,"Slice of Life, Comedy, Drama, School, Shounen",8.7,0.500000,0.8,0.4
258,523,Tonari no Totoro,My Neighbor Totoro,"Adventure, Comedy, Supernatural",8.31,1.000000,0.4,0.4
1260,10800,Chihayafuru,Chihayafuru,"Drama, Game, Josei, School, Slice of Life, Sports",8.23,1.000000,0.4,0.4
201,403,Hanaukyou Maid-tai,Unknown,"Comedy, Ecchi, Harem, Romance",6.62,0.666667,0.6,0.4


## Evaluacion de Performance

### Matriz Confusion

In [ ]:

# Esta funcion va a calcular la matriz de confusion
# Para esto se dividira un 40% de los animes que el usuario a visto para evaluar el modelo. Luego,
# mirando la interseccion entre las recomendaciones generadas y y este conjunto de evaluacion se calcula la 
# matriz de confusion.

def conf_Matrix_KNN_recomender(user_total, X_train, MAL_IDs,k_neighbors = 10):
    # X_train has liked (1) and disliked (-1) and not watched (0), lo mismo para user_total
    X_v = X_train[:,0].reshape(-1,1)
    user_v = np.array([user_total[0]])
    # user_score = []
    indexes = []
    for i in range(0, len(X_train[0,:])):
        if user_total[i]!=0:
            viewed.append(MAL_IDs[i])
            # user_v = np.append(user_v, user_total[i])
            # X_v = np.concatenate((X_v, X_train[:,i].reshape(-1,1)), axis=1)
            indexes.append(i)
    
    # user_v = user_v[1:] 
    # X_v = X_v[:,1:]
    shuffle(indexes)
    indexes_train = indexes[:int(np.floor(len(indexes)*0.6))]
    indexes_test = indexes[int(np.floor(len(indexes)*0.6)):]
    
    viewed_train = []
    viewed_test = []
    X_v_train = X_train[:,0].reshape(-1,1)
    user_v_train = np.array([user_total[0]])
    X_v_test = X_train[:,0].reshape(-1,1)
    user_v_test = np.array([user_total[0]])
    for i in indexes_train:
            viewed_train.append(MAL_IDs[i])
            user_v_train = np.append(user_v_train, user_total[i])
            X_v_train = np.concatenate((X_v_train, X_train[:,i].reshape(-1,1)), axis=1)
            
    for i in indexes_test:
            viewed_test.append(MAL_IDs[i])
            user_v_test = np.append(user_v_test, user_total[i])
            X_v_test = np.concatenate((X_v_test, X_train[:,i].reshape(-1,1)), axis=1)
            
    user_v_train = user_v_train[1:] 
    X_v_train = X_v_train[:,1:]
    user_v_test = user_v_test[1:] 
    X_v_test = X_v_test[:,1:]

    model_knn = NearestNeighbors(
              # metric='cosine',
              n_neighbors=k_neighbors)
    model_knn.fit(X_v_train)

    dist, knn_index = model_knn.kneighbors(user_v_train.reshape(1, -1) , return_distance=True) # esto me devuelve los indices de los K samples mas cercanos
    knn_index = knn_index[0]
    # knn_index
    recomendations = []
    recom_value = []
    count = []
    user_v_eval = []
    for kn in knn_index:
        for i in range(1, len(X_v_test[0,:])):
            if X_v_test[kn,i]!=0: # and (MAL_IDs[i] not in viewed):
                if str(i) not in recomendations:
                    recomendations.append(MAL_IDs[indexes_test[i]])
                    recom_value.append(X_train[kn,i])
                    count.append(1)
                    user_v_eval.append(user_v_test[i])
                else:
                  # if i in recomendations:
                    recom_value[np.where(np.array(recomendations)==str(i))[0][0]] += X_train[kn,i]#/k_neighbors
                    count[np.where(np.array(recomendations)==str(i))[0][0]] += 1
    # score_viewed = 0
    score = 0
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    # print(user_v_eval)
    if user_v_eval!=[]:
        for i in range(len(recomendations)):
            if recom_value[i]!=0:
                # print(recom_value[i])
                # recomendations[i] = int(recomendations[i])
                score = recom_value[i]*user_v_eval[i]#/(k_neighbors*len(user_v_eval))
                # print(score)
                if score>0:
                    if user_v_eval[i]>0:
                      tp+=1
                    else:
                        tn+=1
                else:
                    if user_v_eval[i]>0:
                        fn+=1
                    else:
                        fp+=1
    return [[tp, fp],[fn,tn]]

accept = 0
for i in range(len(X_test[3,1:])):
    if X_test[3,1+i]!=0:
        accept+=1
        # print(X_test[3,1+i])
if accept!=0:
    print(conf_Matrix_KNN_recomender(X_test[3,1:], X_train[:,1:], df.columns))

[[37, 4], [2, 0]]


In [ ]:
# score_v = 0
# score_a = 0
[[tp, fp],[fn,tn]] = [[0,0],[0,0]]
for i  in range(X_test.shape[0]*0+200):
    accept = 0
    for j in range(len(X_test[3,1:])):
        if X_test[i,1+j]!=0:
            accept+=1
    if accept!=0:
        try:
            [[tp_t, fp_t],[fn_t,tn_t]] = conf_Matrix_KNN_recomender(X_test[i,1:], X_train[:,1:], df.columns)
            pass
        except:
            print('ERROR in ', i, end='\t')
        [[tp, fp],[fn,tn]] = [[tp+tp_t, fp+fp_t],[fn+fn_t, tn+tn_t]]
        # score_v += score_v_tr/X_test.shape[0]
        # score_a += score_a_tr/X_test.shape[0]
    #     pass
    # except:
    #     print('ERROR in index ', i)

print()
print([[tp, fp],[fn,tn]])
# (X_test[n,1:])
# X_test.shape[0]

print('BA = ', (tp/(tp+fn) + tn/(tn+fp))/2) #BA
print('Sensibilidad = ', tp/(tp+fn)) # Sensibilidad
print('Especificidad = ', tn/(tn+fp)) #Especificidad
print('Accuracy = ', (tn+tp)/(tn+tp+fn+fp)) #Acc
print('Precision = ', (tp)/(tp+fp))

ERROR in  173	
[[7337, 755], [543, 75]]
BA =  0.5107264081707541
Sensibilidad =  0.9310913705583757
Especificidad =  0.09036144578313253
Accuracy =  0.8509758897818599
Precision =  0.9066979733069699


In [ ]:
# [[tp, fp],[fn,tn]] = [[5553, 1188], [1352, 413]]

print('BA = ', (tp/(tp+fn) + tn/(tn+fp))/2) #BA
print('Sensibilidad = ', tp/(tp+fn)) # Sensibilidad
print('Especificidad = ', tn/(tn+fp)) #Especificidad
print('Accuracy = ', (tn+tp)/(tn+tp+fn+fp)) #Acc
print('Precision = ', (tp)/(tp+fp))

BA =  0.5217793495113496
Sensibilidad =  0.9313868613138686
Especificidad =  0.11217183770883055
Accuracy =  0.8420915712799167
Precision =  0.8955642897248737
